### Задачи к Лекции 4

__Исходные данные__

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/mlbootcamp5_train.csv",
                 sep=";",
                 index_col="id")
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

Mounted at /content/drive


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,True,False,False,True,False,False,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,False,False,True,True,False,False,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,False,False,True,True,False,False,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,True,False,False,True,False,False,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,True,False,False,True,False,False,0


## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

In [10]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss

#Подготовка данных
features = ['age', 'weight', 'height', 'ap_lo', 'ap_hi']
target = 'cardio'

X = df[features]
y = df[target]

#Тренировочные и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

#Обучение моделец
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

#Наивный Байес
nb = GaussianNB()
nb.fit(X_train, y_train)

#Решающее дерево
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

#Предсказание оценки

models = {
    'kNN': knn,
    'Naive Bayes': nb,
    'Decision Tree': dt
}

results = {}

for name, model in models.items():
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]  # Вероятности положительного класса

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)
    loss = log_loss(y_test, y_proba)

    results[name] = {
        'Accuracy': acc,
        'F1-score': f1,
        'ROC AUC': roc_auc,
        'Log Loss': loss
    }

#результаты

df_results = pd.DataFrame(results).T
print(df_results)


best_metrics = df_results.idxmax()
print("\nЛучший классификатор по каждой метрике:")
print(best_metrics)

               Accuracy  F1-score   ROC AUC   Log Loss
kNN            0.679143  0.672149  0.723429   2.365549
Naive Bayes    0.541238  0.213551  0.667530   0.785127
Decision Tree  0.617619  0.617619  0.617655  13.769303

Лучший классификатор по каждой метрике:
Accuracy              kNN
F1-score              kNN
ROC AUC               kNN
Log Loss    Decision Tree
dtype: object


**Комментарии:** Ваши комментарии здесь.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# Загружаем данные
df = pd.read_csv("/content/drive/MyDrive/mlbootcamp5_train.csv", sep=";", index_col="id")
# Используем только указанные признаки
features = ['age', 'weight', 'height', 'ap_lo', 'ap_hi']
X = df[features]
y = df['cardio']

# Предварительная обработка
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Разбиение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Инициализация моделей
models = {
    "kNN": KNeighborsClassifier(n_neighbors=3),
    "Naive Bayes": GaussianNB(),
    "Decision Tree": DecisionTreeClassifier(max_depth=5)
}

# Словарь для результатов
results = {name: {} for name in models.keys()}

# Оценка моделей
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    results[name]['Accuracy'] = accuracy_score(y_test, y_pred)
    results[name]['F1 Score'] = f1_score(y_test, y_pred)
    results[name]['ROC AUC'] = roc_auc_score(y_test, y_proba)
    results[name]['Log Loss'] = log_loss(y_test, y_proba)

# Вывод результатов
for metric in ['Accuracy', 'F1 Score', 'ROC AUC', 'Log Loss']:
    print(f"\n{metric}:")
    for model in results:
        print(f"{model}: {results[model][metric]}")


Accuracy:
kNN: 0.6531904761904762
Naive Bayes: 0.5644285714285714
Decision Tree: 0.7239523809523809

F1 Score:
kNN: 0.6492655911389357
Naive Bayes: 0.3229960772703723
Decision Tree: 0.7135728049804833

ROC AUC:
kNN: 0.6883581178475942
Naive Bayes: 0.6805225766425314
Decision Tree: 0.7877830495965391

Log Loss:
kNN: 4.2084406888909704
Naive Bayes: 0.7074146349487408
Decision Tree: 0.5545822905248292


**Комментарии:** Ваши комментарии здесь.